# Nova Prompt Optimizer

Nova Prompt Optimizer automatically optimizes your prompts for Amazon Nova models using your own datasets. This revolutionary solution transforms the lengthy manual migration process into an efficient, data-driven workflow—helping you unlock Nova's full potential faster than ever before.

### Install the NovaPromptOptimizer SDK

pip install the SDK

In [ ]:
import sys
!{sys.executable} -m pip install nova-prompt-optimizer

## Section 2: Initialize the Input Adapters

![adapters](nova_prompt_optimizer/docs/adapters.png)

### Dataset Adapter

Initialize the Dataset Adapter that takes the input_columns and output_columns. We use the JSONDatasetAdapter to read a `.jsonl` file and adapt it to the standardized format. Split the dataset into training and testing sets

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.dataset_adapter import JSONDatasetAdapter

input_columns = {"input"}
output_columns = {"answer"}

dataset_adapter = JSONDatasetAdapter(input_columns, output_columns)

# Adapt
dataset_adapter.adapt("nova_prompt_optimizer/data/FacilitySupportAnalyzer.jsonl")

# Create Train and Test Splits. Train Set is used for Optimization, Test Set is used for Evaluation
train_set, test_set = dataset_adapter.split(0.5)

### Prompt Adapter

Initialize the Prompt Adapter for the Original Prompt. For this example, we use the FacilitySupportAnalyzer Original Prompt in the `.txt` format

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter import TextPromptAdapter

prompt_variables = input_columns

prompt_adapter = TextPromptAdapter()

prompt_adapter.set_user_prompt(file_path="nova_prompt_optimizer/original_prompt/user_prompt_template.txt", variables=prompt_variables)

# Adapt
prompt_adapter.adapt()

### Metric Adapter

Initialize the Metric Adapter for evaluating this prompt for certain optimizers. For this example, we build a Custom Metric for the Facility Support Analyzer Dataset. The metric adapter requires the use of the `apply` [For single row evaluation] or `batch_apply` [For evaluating the whole dataset together] function

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.metric_adapter import MetricAdapter
from typing import List, Any, Dict
import re
import json

class FacilitySupportAnalyzerMetric(MetricAdapter):
    def parse_json(self, input_string: str):
        """
        Attempts to parse the given string as JSON. If direct parsing fails,
        it tries to extract a JSON snippet from code blocks formatted as:
            ```json
            ... JSON content ...
            ```
        or any code block delimited by triple backticks and then parses that content.
        """
        try:
            return json.loads(input_string)
        except json.JSONDecodeError as err:
            error = err

        patterns = [
            re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE),
            re.compile(r"```(.*?)```", re.DOTALL)
        ]

        for pattern in patterns:
            match = pattern.search(input_string)
            if match:
                json_candidate = match.group(1).strip()
                try:
                    return json.loads(json_candidate)
                except json.JSONDecodeError:
                    continue

        raise error

    def _calculate_metrics(self, y_pred: Any, y_true: Any) -> Dict:
        strict_json = False
        result = {
            "is_valid_json": False,
            "correct_categories": 0.0,
            "correct_sentiment": False,
            "correct_urgency": False,
        }

        try:
            y_true = y_true if isinstance(y_true, dict) else (json.loads(y_true) if strict_json else self.parse_json(y_true))
            y_pred = y_pred if isinstance(y_pred, dict) else (json.loads(y_pred) if strict_json else self.parse_json(y_pred))
        except json.JSONDecodeError:
            result["total"] = 0
            return result  # Return result with is_valid_json = False
        else:
            result["is_valid_json"] = True

            categories_true = y_true.get("categories", {})
            categories_pred = y_pred.get("categories", {})

            if isinstance(categories_true, dict) and isinstance(categories_pred, dict):
                correct = sum(
                    categories_true.get(k, False) == categories_pred.get(k, False)
                    for k in categories_true
                )
                result["correct_categories"] = correct / len(categories_true) if categories_true else 0.0
            else:
                result["correct_categories"] = 0.0  # or raise an error if you prefer

            result["correct_sentiment"] = y_pred.get("sentiment", "") == y_true.get("sentiment", "")
            result["correct_urgency"] = y_pred.get("urgency", "") == y_true.get("urgency", "")

        # Compute overall metric score
        result["total"] = sum(
            float(result[k]) for k in ["correct_categories", "correct_sentiment", "correct_urgency"]
        ) / 3.0

        return result

    def apply(self, y_pred: Any, y_true: Any):
        return self._calculate_metrics(y_pred, y_true)

    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        evals = [self.apply(y_pred, y_true) for y_pred, y_true in zip(y_preds, y_trues)]
        float_keys = [k for k, v in evals[0].items() if isinstance(v, (int, float, bool))]
        return {k: sum(e[k] for e in evals) / len(evals) for k in float_keys}

metric_adapter = FacilitySupportAnalyzerMetric()

### Inference Adapter
Initialize the InferenceAdapter to choose the backend Inference. Currently, we only support BedrockInferenceAdapter.

In [ ]:
from amzn_nova_prompt_optimizer.core.inference.adapter import BedrockInferenceAdapter

inference_adapter = BedrockInferenceAdapter(region_name="us-west-2")

## Section 3: Evaluate the Original Prompt

### Evaluator

The Evaluator can use the metric_adapter, prompt_adapter, and dataset_adapter to evaluate the prompt given the `model_id` to produce an evaluation score. The Evaluator internally uses the `InferenceRunner` to first generate inference results and then evaluate the output.

#### Base Model Evaluation

In [ ]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(prompt_adapter, test_set, metric_adapter, inference_adapter)

In [ ]:
original_prompt_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

print(f"Original Prompt Evaluation Score = {original_prompt_score}")

## Section 4: Optimize the Prompt

### Create a Numerical Metric using the Custom Metric

Optimization run requires numerical metrics to quantify the optimization and produce the best results. We slightly update our metrics `apply` function to provide us a numerical value.

In [ ]:
class FacilitySupportAnalyzerNovaPromptOptimizerMetric(FacilitySupportAnalyzerMetric):
    def apply(self, y_pred: Any, y_true: Any):
        # Requires to return a value and not a JSON payload
        return self._calculate_metrics(y_pred, y_true)["total"]
        
    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        pass
    
nova_prompt_optimizer_metric_adapter = FacilitySupportAnalyzerNovaPromptOptimizerMetric()

### Optimization Adapter

We can now define the Optimization Functions. The Optimization function takes as input the Prompt Adapter and Inference Adapter and Optionally a Dataset Adapter and Metric Adapter. The optimization function optimizes the prompt and returns a Prompt Adapter.

### Nova Prompt Optimizer

NovaPromptOptimizer is a combination of Meta Prompting using the Nova Guide on prompting and DSPy's MIPROv2 Optimizer using Nova Prompting Tips. NovaPromptOptimizer first runs a meta prompter to identify system instructions and user template from the prompt adapter. Then MIPROv2 is run on top of this to optimize system instructions and identify few-shot samples that need to be added. The few shot samples are added as converse format so they are added as User/Assistant turns.

**Requirements:** NovaPromptOptimizer requires Prompt Adapter, Dataset Adapter, Metric Adapter and Inference Adapter.

In [ ]:
from amzn_nova_prompt_optimizer.core.optimizers import NovaPromptOptimizer

# We provide the train_set here
nova_prompt_optimizer = NovaPromptOptimizer(prompt_adapter=prompt_adapter, inference_adapter=inference_adapter, dataset_adapter=train_set, metric_adapter=nova_prompt_optimizer_metric_adapter)

# Since we are using Nova Lite, we will use "lite" mode for optimization
optimized_prompt_adapter = nova_prompt_optimizer.optimize(mode="lite")

### Optimized System Prompt

In [ ]:
optimized_prompt_adapter.system_prompt

### Optimized User Prompt

In [ ]:
optimized_prompt_adapter.user_prompt

### Few Shot Examples

In [ ]:
print(f"Number of Few-Shot Examples = {len(optimized_prompt_adapter.few_shot_examples)}")

In [ ]:
# Print only the first example
print(optimized_prompt_adapter.few_shot_examples[0])

### Save the Prompt

In [ ]:
optimized_prompt_adapter.save("nova_prompt_optimizer/optimized_prompt/")

## Section 5: Evaluate the Optimized Prompt

### Evaluator

Now we evaluate the Nova Prompt Optimizers Optimized prompt

In [ ]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(optimized_prompt_adapter, test_set, metric_adapter, inference_adapter)

In [ ]:
nova_prompt_optimizer_eval_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

In [ ]:
print(f"Optimized Prompt Evaluation Score = {nova_prompt_optimizer_eval_score}")

### Save your Evaluation Results

In [ ]:
evaluator.save("nova_prompt_optimizer/evals/nova_lite/nova_prompt_optimizer_eval.jsonl")

# Summary

Congratulations! You have completed the workshop on Nova Prompt Optimization using our new Nova Prompt Optimizer SDK

## Key Concepts Covered

## Next Steps